### Small Cap Value and Growth

- small cap $\approx$ Russell 2000
- value usually measured by PB or PE
- some academic work (Fama-French) found PB is a better predictor of returns
- low PB = value, high PB = growth
- academics usually use BP instead of PB and call it book-to-market
- high BP = value, low BP = growth
- small-cap growth has historically had very poor returns

### Value and Momentum (Part 1)

- get marketcap data in addition to prices
- calculate momentum
- keep stocks between 1,001 and 3,000 in market cap
- create 5x5 sort on value and momentum
- compute equally weighted portfolio returns

### Value and Momentum (Part 2)

- rank each stock between 1,001 and 3,000 on value and momentum
- high rank = high value or high momentum (good)
- add ranks to get a single score
- go long top 50 and short bottom 50
- compute long minus short returns

### Value and Momentum (Part 3)

- For long only portfolio, choose best stocks in each sector and match sector weights to benchmark (e.g., Russell 2000).
- For long-short portfolio, match shorts and longs in each sector to get market-neutral and sector-neutral portfolio.

### Value and Momentum and Other Things (Part 4)

- Use machine learning to find the optimal way to combine value and momentum
- And add other predictors (ROE, investment rate, short-term reversal, ...)

### Data and Procedure

- Get sectors from tickers table
- Get marketcap and pb from weekly table
- Get closeadj and closeunadj from sep_weekly as before
- Calculate momentum as before
- Filter to 1,001-3,000 on marketcap each week
- Form portfolios

### Create connection

In [2]:
from sqlalchemy import create_engine
import pymssql
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

### Get data

In [4]:
import pandas as pd 
tickers = pd.read_sql(
    """ 
    select ticker, sector from tickers
    """,
    conn
)
tickers.head(3)

,ticker,sector
0,A,Healthcare
1,AA,Basic Materials
2,AAAB,Financial Services


In [11]:
weekly = pd.read_sql(
    """ 
    select ticker, date, marketcap, pb, lastupdated
    from weekly
    where date>='2010-01-01'
    order by ticker, date, lastupdated
    """,
    conn
)
weekly = weekly.groupby(["ticker", "date", "lastupdated"]).last()
weekly = weekly.droplevel("lastupdated")
weekly.head(3)

marketcap   pb
ticker date                      
A      2010-01-08    10918.4  4.4
       2010-01-15    10751.0  4.3
       2010-01-22    10744.0  4.3

In [8]:
df = pd.read_sql(
    """ 
    select ticker, date, closeadj, closeunadj, lastupdated 
    from sep_weekly 
    where date>='2010-01-01'
    order by ticker, date, lastupdated    
    """,
    conn,
)
df = df.groupby(["ticker", "date", "lastupdated"]).last()
df = df.droplevel("lastupdated")

### Calculate momentum and throw out penny stocks

In [9]:
df["ret"] = df.groupby("ticker", group_keys=False).closeadj.pct_change()
df["adj_shift"] = df.groupby("ticker", group_keys=False).closeadj.shift()
df["annual"] = df.groupby("ticker", group_keys=False).adj_shift.pct_change(52)
df["monthly"] = df.groupby("ticker", group_keys=False).adj_shift.pct_change(4)
df["mom"] = df.groupby("ticker", group_keys=False).apply(
    lambda d: (1+d.annual)/(1+d.monthly) - 1
)
df["unadj_shift"] = df.groupby("ticker", group_keys=False).closeunadj.shift()
df = df[df.unadj_shift > 5]
df = df[["ret", "mom"]].dropna().reset_index().copy()
df.head(3)

,ticker,date,ret,mom
0,A,2011-01-14,0.008130,0.199287
1,A,2011-01-21,0.050456,0.270914
2,A,2011-01-28,-0.075973,0.337839


### Merge marketcap

In [12]:
weekly = weekly.groupby("ticker", group_keys=False)[["marketcap", "pb"]].shift().dropna()
weekly = weekly.reset_index()
weekly.head(3)

,ticker,date,marketcap,pb
0,A,2010-01-15,10918.4,4.4
1,A,2010-01-22,10751.0,4.3
2,A,2010-01-29,10744.0,4.3


In [13]:
df = df.merge(weekly, on=["ticker", "date"], how="inner")
df.head(3)

,ticker,date,ret,mom,marketcap,pb
0,A,2011-01-14,0.008130,0.199287,14557.7,4.5
1,A,2011-01-21,0.050456,0.270914,14675.8,4.5
2,A,2011-01-28,-0.075973,0.337839,15416.2,4.8


### Filter to small caps